In [ ]:
!pip3 install Pillow
!pip3 install tensorflow_hub
!pip3 install pandas
!pip3 uninstall tensorflow -y
!pip3 install tensorflow-gpu

In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import pandas as pd
import gc

In [ ]:
ROOT = '/content/drive/MyDrive/ml/hacks/avito/data/'
IMG_HEIGHT, IMG_WIDTH = 224, 224
batch_size=300
df_columns = ['image_id'] + ['image_%i' % i for i in range(2048)]
tf.config.list_physical_devices('GPU')

In [ ]:
def process_path(file_path):
  img = tf.io.read_file(file_path)
  if tf.io.is_jpeg(img):
    img = tf.cast(tf.io.decode_jpeg(img, channels=3), tf.uint8)
  else:
    img = tf.cast(tf.zeros((IMG_HEIGHT, IMG_WIDTH, 3)), tf.uint8)
  img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])
  return img, tf.strings.regex_replace(tf.strings.split(file_path, os.path.sep)[-1], '\.jpg', '')

NameError: ignored

In [ ]:
Image.fromarray(np.zeros((IMG_WIDTH, IMG_HEIGHT, 3), dtype=np.uint8)).save(ROOT + 'images/empty.jpg')

In [ ]:
offset = 0
num_images = 1390836
data = tf.data.Dataset.list_files(ROOT + 'images/*.jpg', shuffle=False).skip(offset)
data = data.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
data = data.batch(batch_size)
data = data.prefetch(buffer_size=tf.data.AUTOTUNE)

mobilenet_v2 = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5"
feature_extractor_layer = hub.KerasLayer(
    mobilenet_v2,
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
    trainable=False)

values = []
for i, (batch, label) in enumerate(data):
    batch_arr = feature_extractor_layer(batch).numpy()
    label_arr = label.numpy().astype(np.dtype('U', 8))
    values += np.hstack([label_arr.reshape(label_arr.shape[0], 1), batch_arr]).tolist()
    print("Elements processed - {}".format(offset + (i + 1) * batch_size))
    if i % 100 == 0:
      df = pd.DataFrame(values, columns=df_columns)
      header = True if i == 0 and offset == 0 else False
      mode = 'w' if i == 0 and offset == 0 else 'a'
      df.to_csv(ROOT + 'images_resnet.csv', header=header, mode=mode, index=False)
      values = []
      df = None
      gc.collect()
if len(values) > 0:
    pd.DataFrame(values, columns=df.columns).to_csv(ROOT + 'images_resnet.csv', header=False, mode='a', index=False)
    values = []